In [ ]:
import h5py

# Ouvrir le fichier h5
file_path = 'chemin/vers/votre/fichier.h5'
h5_file = h5py.File(file_path, 'r')

# Accéder aux groupes de données
analysis_group = h5_file['analysis']
metadata_group = h5_file['metadata']

# Extraire certaines caractéristiques
chroma_features = analysis_group['segments_pitches'][:]
mfccs = analysis_group['segments_timbre'][:]
tempo = analysis_group['tempo'][()]
key = analysis_group['key'][()]
loudness = analysis_group['loudness'][()]
# ... et ainsi de suite

# Fermer le fichier h5
h5_file.close()


In [ ]:
import h5py
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models

# Charger les données à partir du fichier h5
file_path = 'chemin/vers/votre/fichier.h5'
h5_file = h5py.File(file_path, 'r')
segments_pitches = h5_file['analysis']['segments_pitches'][:]
segments_timbre = h5_file['analysis']['segments_timbre'][:]
loudness = h5_file['analysis']['loudness'][:]
labels = h5_file['metadata']['genre'][:]  # Assurez-vous d'avoir les étiquettes de genre dans votre fichier

# Fermer le fichier h5
h5_file.close()

# Prétraitement des données
# Concaténer les caractéristiques segments_pitches et segments_timbre
features = np.concatenate((segments_pitches, segments_timbre), axis=1)

# Normalisation des caractéristiques
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Encodage des étiquettes de genre en nombres
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels_encoded, test_size=0.2, random_state=42)

# Création du modèle
model = models.Sequential([
    layers.Input(shape=(features_scaled.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(label_encoder.classes_), activation='softmax')  # Couche de sortie
])

# Compilation du modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')


In [ ]:
import librosa
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras import layers, models

# Charger le modèle préalablement entraîné
model = tf.keras.models.load_model('chemin/vers/votre/modele.h5')  # Remplacez par le chemin de votre modèle

# Charger le fichier audio .wav
wav_file = 'chemin/vers/votre/fichier.wav'  # Remplacez par le chemin de votre fichier .wav
y, sr = librosa.load(wav_file)

# Extraire les caractéristiques (segments_pitches et segments_timbre)
chroma_features = librosa.feature.chroma_stft(y=y, sr=sr)
mfccs = librosa.feature.mfcc(y=y, sr=sr)
segments_features = np.concatenate((chroma_features, mfccs), axis=1)

# Normalisation des caractéristiques
scaler = StandardScaler()
segments_features_scaled = scaler.fit_transform(segments_features)

# Faire une prédiction sur le modèle
predicted_probs = model.predict(np.expand_dims(segments_features_scaled, axis=0))
predicted_label_index = np.argmax(predicted_probs)
confidence = predicted_probs[0][predicted_label_index]

# Charger l'encodeur d'étiquettes
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('chemin/vers/votre/label_encoder.npy')  # Remplacez par le chemin de votre encodeur d'étiquettes

# Obtenir le genre prédit
predicted_genre = label_encoder.inverse_transform([predicted_label_index])[0]

print(f'Predicted genre: {predicted_genre} (Confidence: {confidence:.2f})')
